# ETL DataSet Train -  EEGSA - Guatemala

**Leer los datos en crudo (raw data) procesarlos, guardarlos (clean data) y generar data set de entrenamiento**

In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
import pandas as pd
import io
import glob
import unidecode
from tqdm import tqdm
import sys

In [ ]:
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format #evita que muestre notacion cientifica
pd.set_option('display.max_columns', None)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [11]:
from src.etl.etl_train import *
from src.conf.config  import ConfigETL

### Configuracion

In [12]:
ConfigETL.init_config()

In [13]:
DATA_PATH_RAW = ConfigETL.get('workspace').get('data_raw_path')
DATA_PATH_CLEAN = ConfigETL.get('workspace').get('data_clean_path')

file_name_historico_consumo = ConfigETL.get('output_data_train').get('file_name_historico_consumo')
file_name_historico_ordenes = ConfigETL.get('output_data_train').get('file_name_historico_ordenes')
file_name_extra_data = ConfigETL.get('output_data_train').get('file_name_extra_data')
file_name_data_train = ConfigETL.get('output_data_train').get('file_name_data_train')

In [14]:
def save_file(df,filename):
    path = DATA_PATH_CLEAN+filename
    df.to_parquet(path, index=False)
        
def load_file(filename):
    path = DATA_PATH_CLEAN+filename
    return pd.read_parquet(path)
        
def run_etl():
    df = run_etl_historico_consumo(DATA_PATH_RAW)
    save_file(df,file_name_historico_consumo)
    
    df = run_etl_historico_ordenes(DATA_PATH_RAW)
    save_file(df,file_name_historico_ordenes)
    
    df = run_etl_zgm023(DATA_PATH_RAW)
    save_file(df,file_name_extra_data)
    
def run():
    run_etl()
    df_consumo = load_file(file_name_historico_consumo)
    df_ordenes = load_file(file_name_historico_ordenes)
    df_exdata = load_file(file_name_extra_data)
    df_train = create_train_dataset(df_consumo,df_ordenes,df_exdata)
    save_file(df_train,file_name_data_train)

In [ ]:
%%time
run()

In [15]:
df_consumo = load_file(file_name_historico_consumo)

In [16]:
df_consumo.head()

,id_usuario,departamento,municipio,zona,mes_operacion,consumo,tipo_tarifa,year,mes,date
0,2,escuintla,palin,zona_3,202107,40.00000,TS,2021,07,2021-07-01
1,5,escuintla,palin,zona_3,202107,166.00000,TS,2021,07,2021-07-01
2,7,escuintla,palin,zona_3,202107,58.00000,TS,2021,07,2021-07-01
3,12,escuintla,palin,zona_3,202107,264.00000,TS,2021,07,2021-07-01
4,13,escuintla,palin,zona_3,202107,142.00000,TS,2021,07,2021-07-01


In [17]:
df_ordenes = load_file(file_name_historico_ordenes)

In [18]:
df_ordenes.head()

,clase,subclase,orden,cod_mat,contrato,codigo_postal,tecnico,f_ejec,c_e,file_year,has_anomalia,no_ejecuto,cant_filas,is_fraud,mes,year,date
0,ircs,iu,561378946,41-0084,0000067328,1015,energ-04,2017-10-02,n11,2017,0,0,1,0,10,2017,2017-10-01
1,ircs,iu,561381828,41-0082,0000095801,5009,u_inspec,2017-10-12,n34,2017,0,0,1,0,10,2017,2017-10-01
2,ircs,an,561378788,41-0082,0000284313,1064,gauss-35,2017-10-02,n08,2017,1,0,1,1,10,2017,2017-10-01
3,ircs,iu,561376419,41-0082,0000304946,1003,gauss-32,2017-09-25,n28,2017,0,0,1,0,9,2017,2017-09-01
4,iprs,pr,550386359,41-0083,0000454088,1057,u_inspec,2017-09-19,n21,2017,0,0,1,0,9,2017,2017-09-01


In [19]:
df_exdata = load_file(file_name_extra_data)

In [21]:
df_exdata.head()

,unidad_de_lectura,contrato,instalacion,codigo_postal,fecha_de_alta,fecha_de_baja,medidor,cod_mat,indice_de_solvencia,no_de_poste,tarfia,multiplicador,actividad_economica,kw_cont,kw_max,desocupado,medidor_interior,anomalia,num_de_cargos_por_anomalia,deuda_anomalia,no_orden_3,fecha_3,clase_de_actividad_3_al_cierre_de_orden,no_orden_2,fecha_2,clase_de_actividad_2_al_cierre_de_orden,no_orden_1,fecha_1,clase_de_actividad_1_al_cierre_de_orden,cantidad_de_avisos_acumulados,num_de_aviso_nl_notas_del_lector,aviso_subclase_nl,clase_de_ordenes,cuenta_de_avisos_pf_pr,facturacion_vencida_num,aviso_de_robo_de_medidores_num,consulta_de_fecha_de_lectura_num,consulta_de_saldo_num,cortes_por_falta_de_pago_num,factor_de_estacionalidad,consumo_menos12,consumo_menos11,consumo_menos10,consumo_menos9,consumo_menos8,consumo_menos7,consumo_menos6,consumo_menos5,consumo_menos4,consumo_menos3,consumo_menos2,consumo_menos1,contrato_menos5,fecha_de_alta_menos5,contrato_menos4,fecha_de_alta_menos4,contrato_menos3,fecha_de_alta_menos3,contrato_menos2,fecha_de_alta_menos2,contrato_menos1,fecha_de_alta_menos1,contrato_mas5,fecha_de_alta_mas5,contrato_mas4,fecha_de_alta_mas4,contrato_mas3,fecha_de_alta_mas3,contrato_mas2,fecha_de_alta_mas2,contrato_mas1,fecha_de_alta_mas1,a_e_kwh_menos5,a_e_kwh_menos4,a_e_kwh_menos3,a_e_kwh_menos2,a_e_kwh_menos1,a_e_kwh_mas5,a_e_kwh_mas4,a_e_kwh_mas3,a_e_kwh_mas2,a_e_kwh_mas1,kwh_tarifa_menos5,kwh_tarifa_menos4,kwh_tarifa_menos3,kwh_tarifa_menos2,kwh_tarifa_menos1,kwh_tarifa_mas5,kwh_tarifa_mas4,kwh_tarifa_mas3,kwh_tarifa_mas2,kwh_tarifa_mas1,kwh_promedio_menos5_12meses,kwh_promedio_menos4_12meses,kwh_promedio_menos3_12meses,kwh_promedio_menos2_12meses,kwh_promedio_menos1_12meses,kwh_promedio_mas5_12meses,kwh_promedio_mas4_12meses,kwh_promedio_mas3_12meses,kwh_promedio_mas2_12meses,kwh_promedio_mas1_12meses,longitud_x,latitud_y,tpo_de_aviso,zona,folio,nivel_de_tension,orden_abierta,fecha_generada,saldo_actual,date_filename
0,708-001,1673322,3001496021,1064,2019-03-01,2200-12-31,U-57788,41-0083,0,358945,R11,1,83_009,0.00000,0.00000,No,sin_dato,sin_dato,0,0.00000,sin_dato,00/00/0000,sin_dato,sin_dato,00/00/0000,sin_dato,sin_dato,00/00/0000,sin_dato,0.00000,NaN,sin_dato,sin_dato,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.75000,73.00000,63.00000,55.00000,52.00000,52.00000,54.00000,79.00000,59.00000,54.00000,56.00000,55.00000,59.00000,83003.00000,11/07/2001,83013.00000,11/07/2001,83019.00000,11/07/2001,83026.00000,11/07/2001,1673328.00000,01/03/2019,83077.00000,11/07/2001,1233849.00000,12/07/2012,1264704.00000,15/12/2012,83054.00000,11/07/2001,1820150.00000,14/04/2021,83_007,83_007,83_007,83_007,83_009,83_007,83_009,83_007,83_007,83_007,R11,R11,R11,R11,R11,R11,R11,R11,R11,R11,118.50000,183.75000,128.50000,74.75000,0.33000,83.67000,69.83000,83.92000,58.00000,175.08000,-90.60503,14.53859,sin_dato,zona_2,708-02112-000,2.00000,sin_dato,00/00/0000,0.00000,zgm023/zgm023
1,708-001,1673328,3001496026,1064,2019-03-01,2200-12-31,U-57787,41-0083,18,358945,R11,1,83_009,0.00000,0.00000,No,sin_dato,sin_dato,0,0.00000,sin_dato,00/00/0000,sin_dato,sin_dato,00/00/0000,sin_dato,sin_dato,00/00/0000,sin_dato,0.00000,NaN,sin_dato,sin_dato,0.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.17000,0.00000,0.00000,0.00000,0.00000,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,2.00000,1832277.00000,26/06/2021,83003.00000,11/07/2001,83013.00000,11/07/2001,83019.00000,11/07/2001,83026.00000,11/07/2001,1264704.00000,15/12/2012,83054.00000,11/07/2001,1820150.00000,14/04/2021,NaN,00/00/0000,1673322.00000,01/03/2019,83_009,83_007,83_007,83_007,83_007,83_007,83_007,83_007,0035,83_009,R11,R11,R11,R11,R11,R11,R11,R11,R11,R11,92.17000,118.50000,183.75000,128.50000,74.75000,83.92000,58.00000,175.08000,0.00000,59.25000,-90.60504,14.53854,sin_dato,zona_2,708-02110-000,2.00000,sin_dato,00/00/0000,0.00000,zgm023/zgm023
2,708-001,1694502,3001515067,1064,2019-06-27,2200-12-31,V-18482,41-0083,3,247874,R11,1,83_005,0.00000,0.00000,No,sin_dato,sin_dato,0,0.0000

In [22]:
df_train = load_file(file_name_data_train)

In [23]:
df_train.head()

,id_usuario,12_anterior,11_anterior,10_anterior,9_anterior,8_anterior,7_anterior,6_anterior,5_anterior,4_anterior,3_anterior,2_anterior,1_anterior,date_fizcalizacion,cant_ttarifa,departamento,municipio,zona,tipo_tarifa,contrato,date,cod_mat,is_fraud,cant_null,eliminar,unidad_de_lectura,codigo_postal,fecha_de_alta,no_de_poste,tarfia,multiplicador,actividad_economica,kw_cont,medidor_interior,folio,nivel_de_tension,id
0,100035,153.00000,125.00000,117.00000,120.00000,128.00000,80.00000,105.00000,123.00000,101.00000,111.00000,99.00000,96.00000,2022-01-01,1,guatemala,villa_nueva,zona_3,TS,100035,2022-01-01,41-0069,0,0,0,708-008,1064,2001-07-11,238366,R11,1.00000,83_009,0.00000,sin_dato,708-10329-000,2.00000,0
1,100043,253.00000,246.00000,259.00000,262.00000,254.00000,184.00000,188.00000,189.00000,201.00000,204.00000,189.00000,188.00000,2022-01-01,1,guatemala,guatemala,zona_4,TS,100043,2022-01-01,41-0081,0,0,0,605-015,1004,2001-07-06,135771,C11,1.00000,85_009,0.00000,sin_dato,605-11884-000,2.00000,1
2,10008,187.00000,0.00000,205.00000,187.00000,178.00000,194.00000,200.00000,188.00000,168.00000,0.00000,0.00000,26.00000,2022-01-01,1,guatemala,guatemala,zona_10,TS,10008,2022-01-01,41-0083,0,0,0,603-001,1010,2001-07-04,107349,C11,1.00000,83_002,0.00000,X,603-02018-000,2.00000,2
3,1000912,657.00000,576.00000,636.00000,595.00000,659.00000,598.00000,616.00000,657.00000,613.00000,613.00000,633.00000,616.00000,2022-01-01,1,guatemala,mixco,zona_4,TNS,1000912,2022-01-01,41-0083,0,0,0,612-039,1057,2008-07-24,518933,C11,1.00000,49_005,0.00000,sin_dato,612-53632-000,2.00000,3
4,1000996,70.00000,61.00000,61.00000,69.00000,85.00000,97.00000,96.00000,95.00000,100.00000,104.00000,90.00000,76.00000,2022-01-01,1,guatemala,san_jose_del_golfo,zona_0,TS,1000996,2022-01-01,41-0084,0,0,0,813-035,1053,2008-07-24,386109,R11,1.00000,83_009,0.00000,sin_dato,813-61192-000,2.00000,4
